In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

DATA_FILE = 'CRlncRC/bigtable.txt'
data = pd.read_csv(DATA_FILE, sep='\t')
data


,Gene_ID,adipose_tissue,adrenal_gland,brain,breast,colon,heart,kidney,leukocyte,liver,...,TSS5k_H1hescH3k27ac,TSS5k_H1hescH3k4me1,TSS5k_H1hescH3k4me3,TSS5k_K562H3k27ac,TSS5k_K562H3k4me1,TSS5k_K562H3k4me3,cancer-miRNA_targets,miRNA_targets,hbm_total_degree,hbm_cancer-protein_degree
0,ENSG00000083622,0.00000,0.000,0.00000,0.00000,0.35625,0.158333,0.000,0.000000,0.000000,...,1.26696,1.41547,1.719480,1.42071,8.05691,2.61817,0,0,68,0
1,ENSG00000099869,0.00000,0.000,0.00000,0.11875,0.00000,0.000000,0.000,0.000000,0.316667,...,3.13681,2.72905,1.002450,23.38600,3.95775,2.94355,0,0,168,1
2,ENSG00000115934,0.11875,0.095,0.00000,0.00000,0.11875,0.000000,0.095,0.000000,0.000000,...,0.96861,1.40607,0.997241,1.14729,1.73291,1.32346,0,0,0,0
3,ENSG00000117242,1.18750,1.900,0.83125,1.18750,0.71250,3.166670,0.950,0.678571,1.266670,...,1.02540,1.79405,0.988256,1.18682,1.57390,1.33713,5,6,26,0
4,ENSG00000122043,0.00000,0.000,0.00000,0.00000,0.00000,0.000000,0.000,0.135714,0.000000,...,1.85462,1.71931,1.158130,1.57266,1.45060,1.15152,0,0,328,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11959,ENSG00000273483,0.23750,0.380,0.95000,0.95000,0.23750,0.475000,0.475,0.407143,0.000000,...,23.12970,7.37056,1.730680,2.13592,3.32636,1.57727,0,0,17,0
11960,ENSG00000273486,0.59375,1.900,0.95000,3.56250,0.35625,1.583330,0.475,0.067857,0.316667,...,1.16022,1.50000,6.980600,1.77908,20.13630,4.20219,0,0,52,0
11961,ENSG00000273487,0.11875,0.475,7.12500,0.83125,1.18750,0.633333,0.095,0.135714,0.158333,...,1.78621,1.46059,1.765190,1.73917,2.01012,1.17085,0,0,1991,9
11962,ENSG00000273492,0.11875,0.285,1.18750,0.23750,0.11875,0.316667,0.475,0.000000,0.158333,...,5.33487,39.63620,2.964520,55.11380,12.38110,48.66560,0,0,213,2


In [20]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.metrics import roc_curve

"""Tools for ML training"""


def feature_select(df):
    '''Feature selection based on tree feature importances
    '''
    from sklearn.ensemble import ExtraTreesClassifier
    y = np.array(df.label)
    X=np.array(df.iloc[:,1:].values)
    
    # Build a forest and compute the feature importances
    forest = ExtraTreesClassifier(n_estimators=100,
                                  random_state=0, n_jobs = -1)
    forest.fit(X, y)
    importances = forest.feature_importances_
    indices = np.argsort(importances)[::-1]
    # Print the feature ranking
    
    best_features = ['label']
    for f in range(X.shape[1]):
        best_features.append(df.columns[indices[f]+1])

    n_best = len(best_features)
    df = df.loc[:,best_features[0:n_best]] # 0 = label

    return df


def get_data(path_to_folder):
    """Get the fata from the data file.
    """

    # Reading data
    df = pd.read_csv(os.path.join(path_to_folder, 'bigtable.txt'), sep = '\t', index_col = 0, header = 0)

    # Replacing NaNs
    df = df.fillna(method='pad')

    # Reading negative-label lncRNAs
    with open(os.path.join(path_to_folder, "negative.lncRNA.glist.xls"), "r") as f:
        raw_negative_list = list(map(lambda x: x.strip(), f.readlines()))
        negative_list = list(np.random.choice(raw_negative_list, 150))
    # Reading positive-label lncRNAs
    with open(os.path.join(path_to_folder, "positive.lncRNA.glist.xls"), "r") as f:
        positive_list = list(map(lambda x: x.strip(), f.readlines()))
    
    # Selecting positive and negative samples
    positive_df = df.loc[positive_list,:]
    negative_df = df.loc[negative_list,:]
    positive_df['label'] = 1
    negative_df['label'] = -1
      
    new_df = pd.concat([positive_df, negative_df])

    cols = list(new_df.columns)
    new_cols = cols[:-1]
    new_cols.insert(0,'label')
    new_df = new_df.loc[:,new_cols]
    
    # feature selection
    new_df = feature_select(new_df)
    y = np.array(new_df.label)
    X = new_df.iloc[:,1:]
    
    return X, y


def feature_importance_display(clf, model_id, X):
    if model_id == 'NB':
        importances = pd.DataFrame(
            {
                'positive_odds': clf.feature_log_prob_[0, :],
                'negative_odds': clf.feature_log_prob_[1, :]
            }, 
            index=X.index
        ).sort_values('positive_odds', ascending=False)
    elif model_id == 'LR':
        importances = pd.Series(clf.coef_[0], index=X.columns).sort_values(ascending=True)
    elif model_id == 'RF':
        importances = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=True)
    else:
        raise NotImplementedError(f'The feature importances module is not supporting model_id="{model_id}"..')
    return importances.plot(kind='barh', figsize=(15, 20))


def run_ML_pipeline(report, model_id):
    """
    Runs a certain pipeline on the given training data.
    """

    path_to_folder = 'CRlncRC'
    seed = 42
    np.random.seed(seed)

    train_X, train_y = get_data(path_to_folder)

    if model_id == 'NB':
        from sklearn.naive_bayes import BernoulliNB
        clf = BernoulliNB(alpha = 5)
    
    elif model_id == 'kNN':
        from sklearn.neighbors import KNeighborsClassifier
        clf = KNeighborsClassifier(n_neighbors = 3)
    
    elif model_id == 'LR':
        from sklearn.linear_model import LogisticRegression
        clf = LogisticRegression(C=10, random_state=seed, n_jobs = -1)

    elif model_id == 'RF':
        from sklearn.ensemble import RandomForestClassifier
        clf = RandomForestClassifier(n_estimators=1000, criterion='gini', max_depth=None, \
                                min_samples_split=2,\
                                random_state=seed, n_jobs = -1)

    elif model_id == 'SVM':
        from sklearn.svm import SVC
        clf = SVC(kernel = 'rbf', probability = True, random_state=seed)

    else:
        raise NotImplementedError(f'The model_id={model_id} is not known!')

    clf.fit(train_X, train_y)
    pred_y = clf.predict(train_X)
    
    if report == 'classification_metrics':
        report_dict =  classification_report(train_y, pred_y, output_dict=True)
        df = pd.DataFrame(report_dict).transpose().reset_index()
        return df

    elif report == 'confusion_matrix':
        conf_mat_plot = ConfusionMatrixDisplay.from_estimator(clf, train_X, train_y).ax_
        return conf_mat_plot

    elif report == 'roc_auc_curve':
        roc_auc_curve_plot = RocCurveDisplay.from_estimator(clf, train_X, train_y).ax_
        return roc_auc_curve_plot

    elif report == 'feature_importance':
        feature_importance_plot = feature_importance_display(clf, model_id, train_X)
        return feature_importance_plot

    else:
        raise NotImplementedError(f'The report={report} is not known!')


if __name__ == '__main__':
    run_ML_pipeline('classification_metrics', 'NB')

In [21]:
X, y = get_data('CRlncRC')

In [22]:
X

,genebody_Gm12878H3k4me3,hbm_total_degree,intron_gc,genebody_K562H3k4me1,genebody_SINE,genebody_K562H3k4me3,CDKN2A,genebody_Gm12878H3k27ac,genebody_H1hescH3k4me1,CTNNB1,...,skeletal_muscle_tissue,H3F3A,host_miRNAs,TSS1k_LINE,adipose_tissue,leukocyte,heart,micropeptide_aa_avg,TSS1k_Satellite,genebody_Satellite
Gene_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000172965,3.00769,17,0.445725,11.66760,76,5.61277,0.270973,2.982390,4.09582,0.579457,...,0.000,0.332842,1,1,1.18750,0.678571,0.950000,63.5,0,0
ENSG00000177410,28.75970,91,0.449145,11.76690,6,38.67350,0.685892,11.810900,6.64316,0.397351,...,7.600,0.519883,0,0,26.12500,28.500000,6.333330,0.0,0,0
ENSG00000180139,3.70398,41,0.413681,2.17647,4,2.72839,0.642018,3.975890,3.11366,0.494109,...,0.095,0.263817,0,1,0.83125,0.067857,0.316667,0.0,0,0
ENSG00000186594,13.65620,5,0.641691,13.04190,0,21.61080,-0.108809,25.911700,8.20354,0.539425,...,1.900,-0.185841,1,1,5.93750,1.357140,15.833300,58.0,0,0
ENSG00000188185,4.38400,28,0.437121,2.40041,36,2.30618,0.642910,3.033570,3.19163,0.594239,...,0.095,0.103551,0,1,0.83125,0.339286,0.475000,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000241696,1.83609,5455,0.338293,1.10000,0,1.43731,0.168905,1.128050,2.06472,-0.140131,...,0.000,0.196328,0,0,0.00000,0.000000,0.000000,0.0,0,0
ENSG00000251454,1.94163,5455,0.375837,1.50293,2,1.69148,0.168905,1.071490,1.65424,-0.140131,...,0.000,0.196328,0,1,0.00000,0.000000,0.000000,0.0,0,0
ENSG00000226956,1.74917,4793,0.399663,1.48584,1,2.26685,0.092513,1.206970,4.15427,0.179090,...,0.000,0.113934,0,0,0.00000,0.000000,0.000000,48.0,0,0


In [15]:
a = run_ML_pipeline('classification_metrics', 'NB')

In [16]:
a

,precision,recall,f1-score,support
-1,0.731034,0.706667,0.718644,150.000000
1,0.730061,0.753165,0.741433,158.000000
accuracy,0.730519,0.730519,0.730519,0.730519
macro avg,0.730548,0.729916,0.730039,308.000000
weighted avg,0.730535,0.730519,0.730335,308.000000
